Converting Raw DandiSet to Neural Code

In [1]:
def create_neural_codewords(dandiset_id, filepath, min_unit, max_unit):
    
    # By Colleen Boucher
    #Last edited: 4/10/22
    #INPUT: dandiset_id (str), filepath (str), min_unit (int), max_unit (int)
    #OUTPUT: File named "neural_code_"+dandiset_id + str(datetime.date.today()) +".txt" and saved 
    #        to the current folder
    #SUMMARY: create_neural_codewords is a function that uses the streamed in the Dandiset and converts the 
    #electrophysiology data of the specified range of neurons into neural codewords to be read
    #into the Neural Ideals SageMath package.

    for i in range (min_unit,max_unit):
        unit_number = i
        spike_times = nwb.units["spike_times"][unit_number]
        bin_edges = np.arange(0, 183944, 1/1) #1 = 1 s; 1/100 = cs; 1/1000 = ms time binning
        #183944 s is the total duration of recording that occured across all 3 units for the first
        #data set tried
        #should fix to specify start time as max start time across all units and stop time as min stop 
        #time across all units
    
        binary_spikes, tt = np.histogram(spike_times, bin_edges)

        if i == min_unit:
            matrix_code = binary_spikes
        else:
            matrix_code = np.vstack((matrix_code,binary_spikes))
    
        code_matrix = matrix_code.transpose()
        
    neural_code = ""
    for j in range (0,len(code_matrix)):
        codeword = ""
        for i in range (0,max_unit-min_unit):
            if code_matrix[j][i] != 0:
                code_matrix[j][i] = 1
            codeword = codeword + str(code_matrix[j][i])
        
            if i >= max_unit-min_unit-1:
                codeword = codeword
                neural_code = neural_code + codeword
                break
            
        if j<len(code_matrix)-1:
            neural_code = neural_code + ","
            
    file = open("neural_code_"+dandiset_id + str(datetime.date.today()) +".txt", "w")
    fContent = str(neural_code)

    file.write(fContent)
    file.close()

In [2]:
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
import numpy as np
import matplotlib.pyplot as plt
import datetime

dandiset_id, filepath = "000005", "sub-anm184389/sub-anm184389_ses-20130213_behavior+ecephys.nwb"

with DandiAPIClient() as client:
        asset = client.get_dandiset(dandiset_id, "draft").get_asset_by_path(filepath)
        s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

io = NWBHDF5IO(s3_url, mode='r', load_namespaces=True, driver='ros3')
nwb = io.read()
#nwb2widget(nwb) # Provides tabluar summary of the file's metadata
#useful for determining a specific range of neurons to analyze

min_unit, max_unit = 0, len(nwb.units)
#min_unit and max_unit are to specify a range of neurons desired to be analysed for Dandisets
#to convert the spike train data for a specific range of the neurons in the file use:
#min_unit, max_unit = 22, 31

create_neural_codewords(dandiset_id, filepath, min_unit, max_unit)

### 